## Import libraries

In [1]:
import numpy as np
import librosa as lbr
import librosa.display as display
import wave
import math
import matplotlib.pyplot as plt

## Loop over folders in order to get sample rate and number of frames

In [2]:
folders = ["blues", "classical", "country", "disco", "hiphop", "jazz", "metal", "pop", "reggae", "rock"]

In [3]:
lsr = []
lnf = []

def par(wav):
    try:
        obj = wave.open(wav, 'rb')
        sr = obj.getframerate()
        lsr.append(sr)
        nframes = obj.getnframes()
        lnf.append(nframes)
        obj.close()
    except wave.Error as e:
        print(wav)

In [4]:
pre = "/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original"

for fol in folders:
    for i in range(100):
        if i < 10:
            path = f"{fol}/{fol}.0000{i}.wav"
        else:
            path = f"{fol}/{fol}.000{i}.wav"
        #print(path)
        par(path)

jazz/jazz.00054.wav


In [5]:
unique_sr = list(set(lsr))
print(unique_sr)

unique_nf = list(set(lnf))
print(unique_nf)

[22050]
[661760, 668800, 661504, 663300, 670340, 667920, 669460, 672282, 661408, 660000, 661794, 670120, 663080, 661676, 666160, 667700, 669240, 663740, 665280, 666820, 664400, 665940, 667480, 661980, 663520, 661344, 675808, 665060, 672100, 661100, 668140, 669680, 664180, 661500]


### Il sample rate è uguale per tutti i file.
### Il numero di frames, invece, non è lo stesso, quindi controllo quale
### numero compare maggiormente.

In [6]:
from collections import Counter

frequency_dict = Counter(lnf)
print(frequency_dict)

Counter({661794: 640, 661504: 295, 675808: 10, 661408: 5, 661676: 4, 661760: 4, 669680: 3, 665280: 3, 667920: 3, 669240: 3, 663520: 2, 661344: 2, 663300: 2, 666820: 2, 668140: 2, 670120: 1, 672282: 1, 663080: 1, 668800: 1, 661100: 1, 663740: 1, 664180: 1, 666160: 1, 665060: 1, 664400: 1, 660000: 1, 667700: 1, 665940: 1, 672100: 1, 661980: 1, 667480: 1, 661500: 1, 670340: 1, 669460: 1})


## Modifico la durata dei file con numero frames diverso da 661794

In [10]:
import os
import wave
import numpy as np
from scipy.io import wavfile

def cambia_numero_di_frame(file_input):
    # Imposta il numero di frame desiderato
    numero_frame_desiderato = 661794

    # Crea la cartella "Audio modificati" nella directory corrente se non esiste già
    output_directory = 'Audio modificati'
    os.makedirs(output_directory, exist_ok=True)
    
    # Apri il file WAV e leggi i dati
    sample_rate, audio_data = wavfile.read(file_input)
    
    # Calcola il numero di frame attuale
    numero_frame_attuale = len(audio_data)
    
    # Se il numero di frame è già quello desiderato, non fare nulla
    if numero_frame_attuale == numero_frame_desiderato:
        print(f"Il file {file_input} ha già {numero_frame_desiderato} frame.")
        return
    
    # Ridimensiona i dati audio (interpola per ridurre o aumentare i frame)
    numero_campioni = int(numero_frame_desiderato)
    audio_data_resized = np.interp(
        np.linspace(0, numero_frame_attuale, num=numero_campioni),
        np.arange(numero_frame_attuale),
        audio_data
    ).astype(audio_data.dtype)

    # Salva il file modificato nella cartella "Audio modificati"
    output_path = os.path.join(output_directory, os.path.basename(file_input))
    wavfile.write(output_path, sample_rate, audio_data_resized)
    
    print(f"Audio modificato e salvato in {output_path} con {numero_frame_desiderato} frame.")

In [11]:
for fol in folders:
    for i in range(100):
        if i < 10:
            path = f"{fol}/{fol}.0000{i}.wav"
        else:
            path = f"{fol}/{fol}.000{i}.wav"
        #print(path)
        try:
            cambia_numero_di_frame(path)
        except :
            print(path)

Il file blues/blues.00000.wav ha già 661794 frame.
Il file blues/blues.00001.wav ha già 661794 frame.
Il file blues/blues.00002.wav ha già 661794 frame.
Il file blues/blues.00003.wav ha già 661794 frame.
Il file blues/blues.00004.wav ha già 661794 frame.
Il file blues/blues.00005.wav ha già 661794 frame.
Il file blues/blues.00006.wav ha già 661794 frame.
Il file blues/blues.00007.wav ha già 661794 frame.
Il file blues/blues.00008.wav ha già 661794 frame.
Il file blues/blues.00009.wav ha già 661794 frame.
Il file blues/blues.00010.wav ha già 661794 frame.
Il file blues/blues.00011.wav ha già 661794 frame.
Il file blues/blues.00012.wav ha già 661794 frame.
Il file blues/blues.00013.wav ha già 661794 frame.
Il file blues/blues.00014.wav ha già 661794 frame.
Il file blues/blues.00015.wav ha già 661794 frame.
Il file blues/blues.00016.wav ha già 661794 frame.
Il file blues/blues.00017.wav ha già 661794 frame.
Il file blues/blues.00018.wav ha già 661794 frame.
Il file blues/blues.00019.wav h

In [10]:
hop = 512
frame_size = 2048
sr = 22050
lsp = []

In [11]:
def spec(audio):
    return lbr.stft(audio, n_fft=frame_size, hop_length=hop)

In [12]:
def lib(wav):
    try:
        signal, _ = lbr.load(wav)
        sp = spec(signal)
        lsp.append(sp)
    except :
        print(wav)

In [13]:

for fol in folders:
    print(fol)
    for i in range(100):
        if i < 10:
            path = f"{pre}/{fol}/{fol}.0000{i}.wav"
        else:
            path = f"{pre}/{fol}/{fol}.000{i}.wav"
        lib(path)

blues
classical
country
disco
hiphop
jazz


/tmp/ipykernel_17/563114626.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  signal, _ = lbr.load(wav)
/opt/conda/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/jazz/jazz.00054.wav
metal
pop
reggae
rock
